In [31]:
import pandas as pd
import os

# Load the metadata
maestro_path = "maestro-v3.0.0"
metadata_file = os.path.join(maestro_path, "maestro-v3.0.0.csv")
metadata = pd.read_csv(metadata_file)

# Preview the first few rows
print(metadata.head())


   canonical_composer                canonical_title       split  year  \
0          Alban Berg                   Sonata Op. 1       train  2018   
1          Alban Berg                   Sonata Op. 1       train  2008   
2          Alban Berg                   Sonata Op. 1       train  2017   
3  Alexander Scriabin  24 Preludes Op. 11, No. 13-24       train  2004   
4  Alexander Scriabin               3 Etudes, Op. 65  validation  2006   

                                       midi_filename  \
0  2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...   
1  2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...   
2  2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...   
3  2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...   
4  2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...   

                                      audio_filename    duration  
0  2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...  698.661160  
1  2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...  759.518471  
2  2017/MIDI-Unpr

In [32]:
import pretty_midi
import numpy as np
import json
import os

# File to store normalization parameters
NORMALIZATION_FILE = "output/normalization_params.json"

# Ensure output directory exists
os.makedirs("output", exist_ok=True)

# Load existing normalization data if available
if os.path.exists(NORMALIZATION_FILE):
    with open(NORMALIZATION_FILE, "r") as f:
        normalization_params = json.load(f)
else:
    normalization_params = {"midi_end_times": {}}  # Initialize storage

def extract_features_from_midi(midi_path):
    try:
        # Load MIDI file
        midi = pretty_midi.PrettyMIDI(midi_path)
        midi_end_time = midi.get_end_time()  # Get the end time of the MIDI

        # Get all notes from all instruments (excluding drums)
        notes = []
        for instrument in midi.instruments:
            if not instrument.is_drum:
                for note in instrument.notes:
                    notes.append([
                        note.pitch,  # Pitch
                        note.start,  # Start time (seconds)
                        note.end,    # End time (seconds)
                        note.velocity  # Velocity
                    ])

        if not notes:
            return None  # Skip empty files

        notes = np.array(notes)

        # Normalize features
        min_pitch, max_pitch = 21, 108  # Standard piano range
        notes[:, 0] = (notes[:, 0] - min_pitch) / (max_pitch - min_pitch)  # Normalize pitch
        notes[:, 1] /= midi_end_time  # Normalize start time
        notes[:, 2] /= midi_end_time  # Normalize end time
        notes[:, 3] /= 127  # Normalize velocity

        # Store MIDI end time for denormalization
        midi_filename = os.path.basename(midi_path)
        normalization_params["midi_end_times"][midi_filename] = midi_end_time

        # Save updated normalization parameters
        with open(NORMALIZATION_FILE, "w") as f:
            json.dump(normalization_params, f, indent=4)

        return notes

    except Exception as e:
        print(f"Error processing {midi_path}: {e}")
        return None


In [33]:
import tqdm
import pandas as pd
import os

# Storage for all extracted data
song_dict = {}

# Iterate through all MIDI files
for idx, row in tqdm.tqdm(metadata.iterrows(), total=len(metadata)):
    midi_path = os.path.join(maestro_path, row["midi_filename"])
    
    # Extract features
    features = extract_features_from_midi(midi_path)
    
    if features is not None:
        song_id = f"song_{idx}"
        
        # Convert feature list to DataFrame for easy handling
        song_df = pd.DataFrame(features, columns=["pitch_normalized", "start_normalized", "end_normalized", "velocity_normalized"])
        
        # Assign a unique timestamp index for each note
        song_df["timestamp"] = range(1, len(song_df) + 1)
        
        # Store the data
        song_dict[song_id] = song_df.set_index("timestamp")

# Combine all songs into one DataFrame
df_long = pd.concat(song_dict, names=["song_id", "timestamp"]).reset_index()

# Get the maximum number of timestamps across all songs
max_timestamp = df_long["timestamp"].max()

# Reshape to maintain sequence: pitch1, start1, end1, velo1, pitch2, start2, ...
df_wide = df_long.pivot(index="song_id", columns="timestamp", values=["pitch_normalized", "start_normalized", "end_normalized", "velocity_normalized"])

# Ensure correct ordering
ordered_columns = []
for t in range(1, max_timestamp + 1):  # Iterate up to the max timestamp
    for feat in ["pitch_normalized", "start_normalized", "end_normalized", "velocity_normalized"]:
        if (feat, t) in df_wide.columns:
            ordered_columns.append((feat, t))

# Reorder columns based on the correct sequence
df_wide = df_wide[ordered_columns]

# Fill missing values with default values
df_wide.fillna({"pitch_normalized": 0, "start_normalized": 0.0, "end_normalized": 0.0, "velocity_normalized": 0}, inplace=True)

# Flatten column names to match naming format: pitch1, start1, ...
df_wide.columns = [f"{feat}_{t}" for feat, t in df_wide.columns]

# Reset index
df_wide.reset_index(inplace=True)

# Save to CSV
df_wide.to_csv("output/maestro_transformed.csv", index=False)

print("Data saved in wide format:", df_wide.shape)
print(df_wide.head())


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1276/1276 [04:12<00:00,  5.06it/s]


Data saved in wide format: (1276, 100305)
     song_id  pitch_normalized_1  start_normalized_1  end_normalized_1  \
0     song_0            0.528736            0.001396          0.002571   
1     song_1            0.528736            0.001326          0.002500   
2    song_10            0.402299            0.009004          0.009575   
3   song_100            0.310345            0.004002          0.006236   
4  song_1000            0.528736            0.002598          0.007008   

   velocity_normalized_1  pitch_normalized_2  start_normalized_2  \
0               0.409449            0.586207            0.002534   
1               0.409449            0.586207            0.002217   
2               0.511811            0.494253            0.009207   
3               0.496063            0.724138            0.003911   
4               0.440945            0.482759            0.002611   

   end_normalized_2  velocity_normalized_2  pitch_normalized_3  ...  \
0          0.002708              

In [34]:
data=pd.read_csv("output/maestro_transformed.csv")

In [30]:
data

,song_id,pitch_normalized_1,start_normalized_1,end_normalized_1,velocity_normalized_1,pitch_normalized_2,start_normalized_2,end_normalized_2,velocity_normalized_2,pitch_normalized_3,...,end_normalized_25074,velocity_normalized_25074,pitch_normalized_25075,start_normalized_25075,end_normalized_25075,velocity_normalized_25075,pitch_normalized_25076,start_normalized_25076,end_normalized_25076,velocity_normalized_25076
0,song_0,0.528736,0.001396,0.002571,0.409449,0.586207,0.002534,0.002708,0.527559,0.528736,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,song_1,0.528736,0.001326,0.002500,0.409449,0.586207,0.002217,0.002815,0.472441,0.574713,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,song_10,0.402299,0.009004,0.009575,0.511811,0.494253,0.009207,0.009575,0.511811,0.448276,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,song_100,0.310345,0.004002,0.006236,0.496063,0.724138,0.003911,0.006301,0.661417,0.356322,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,song_1000,0.528736,0.002598,0.007008,0.440945,0.482759,0.002611,0.010268,0.275591,0.425287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,song_995,0.425287,0.001729,0.004229,0.307087,0.459770,0.001693,0.004236,0.377953,0.459770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1272,song_996,0.425287,0.002586,0.004246,0.370079,0.459770,0.002530,0.006064,0.385827,0.425287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1273,song_997,0.425287,0.003506,0.010565,0.307087,0.459770,0.003506,0.011061,0.346457,0.425287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1274,song_998,0.528736,0.003156,0.012272,0.362205,0.482759,0.003215,0.012301,0.354331,0.505747,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
